In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

# from imblearn.combine import SMOTETomek
pd.options.mode.chained_assignment = None

# Load Data

Initial clean

In [85]:
def fill_nan(df, method):
    for col in df.columns:
        
        prc_nan = df[col].isnull().sum() / len(df[col])
        
        if col in ['srch_query_affinity_score']:
            print('Preserved {} with {:.2f} % missing'.format(col, prc_nan * 100))
            continue
        # Impute with median or mean
        if prc_nan < .7:
            print('Preserved {} with {:.2f} % missing'.format(col, prc_nan * 100))
            if method == 'median':
                if col != 'date_time': df[col] = df[col].fillna(df[col].median())
            elif method == 'mean':
                if col != 'date_time': df[col] = df[col].fillna(df[col].mean())
            elif method == 'zero':
                df[col] = df[col].fillna(0)
        # Drop if more than 50% missing
        else:
            print('Dropped {} with {:.2f} % missing'.format(col, prc_nan * 100))
            df = df.drop([col], axis=1)
    return df

def initial_clean(df):
    # We made a boolean variable indicating whether the user performing the query had any purchase history or not. 
    df['srch_hist_bool'] = 0
    df['srch_hist_bool'][~df.visitor_hist_adr_usd.isnull()] = 1
    # Handle missing values
    df[['visitor_hist_adr_usd', 'visitor_hist_starrating']] = df[['visitor_hist_adr_usd', 'visitor_hist_starrating']].fillna(0)
    df_cleaned = fill_nan(df, ' ')
    return df_cleaned

In [52]:
# Clean training set
df = pd.read_csv('../training_set_VU_DM.csv')
df_cleaned = initial_clean(df)
df_cleaned.to_csv('data/training_cleaned.csv')

Preserved srch_id with 0.00 % missing
Preserved date_time with 0.00 % missing
Preserved site_id with 0.00 % missing
Preserved visitor_location_country_id with 0.00 % missing
Preserved visitor_hist_starrating with 0.00 % missing
Preserved visitor_hist_adr_usd with 0.00 % missing
Preserved prop_country_id with 0.00 % missing
Preserved prop_id with 0.00 % missing
Preserved prop_starrating with 0.00 % missing
Preserved prop_review_score with 0.15 % missing
Preserved prop_brand_bool with 0.00 % missing
Preserved prop_location_score1 with 0.00 % missing
Preserved prop_location_score2 with 21.99 % missing
Preserved prop_log_historical_price with 0.00 % missing
Preserved position with 0.00 % missing
Preserved price_usd with 0.00 % missing
Preserved promotion_flag with 0.00 % missing
Preserved srch_destination_id with 0.00 % missing
Preserved srch_length_of_stay with 0.00 % missing
Preserved srch_booking_window with 0.00 % missing
Preserved srch_adults_count with 0.00 % missing
Preserved srch_c

In [53]:
# Clean test set
df = pd.read_csv('../test_set_VU_DM.csv')
df_cleaned = initial_clean(df)
df_cleaned.to_csv('data/test_cleaned.csv')

Preserved srch_id with 0.00 % missing
Preserved date_time with 0.00 % missing
Preserved site_id with 0.00 % missing
Preserved visitor_location_country_id with 0.00 % missing
Preserved visitor_hist_starrating with 0.00 % missing
Preserved visitor_hist_adr_usd with 0.00 % missing
Preserved prop_country_id with 0.00 % missing
Preserved prop_id with 0.00 % missing
Preserved prop_starrating with 0.00 % missing
Preserved prop_review_score with 0.15 % missing
Preserved prop_brand_bool with 0.00 % missing
Preserved prop_location_score1 with 0.00 % missing
Preserved prop_location_score2 with 21.94 % missing
Preserved prop_log_historical_price with 0.00 % missing
Preserved price_usd with 0.00 % missing
Preserved promotion_flag with 0.00 % missing
Preserved srch_destination_id with 0.00 % missing
Preserved srch_length_of_stay with 0.00 % missing
Preserved srch_booking_window with 0.00 % missing
Preserved srch_adults_count with 0.00 % missing
Preserved srch_children_count with 0.00 % missing
Prese

# Further clean

In [3]:
df_train = pd.read_csv('data/training_cleaned.csv')
df_test = pd.read_csv('data/test_cleaned.csv')

# all_list = ['visitor_location_country_id', 'prop_country_id', 'site_id', 'prop_id','prop_starrating', 'prop_review_score', \
#     'prop_location_score1', 'prop_location_score2', 'prop_brand_bool', 'price_usd', 'srch_destination_id', \
#     'orig_destination_distance', 'srch_length_of_stay', 'random_bool', 'promotion_flag']
df_all = pd.concat([df_train, df_test], ignore_index=True)

### One-way count of the feature in the combine of train and test dataset

Conversion of prices

In [4]:
'''
Find outlier of all night price for price_usd
'''
price_list = ['price_usd', 'prop_country_id', 'srch_length_of_stay']
df_price_all = pd.concat([df_train[price_list], df_test[price_list]], ignore_index=True)
df_price_all['price_per_night'] =  df_price_all.price_usd / df_price_all.srch_length_of_stay
df_country_price = df_price_all.groupby('prop_country_id')['price_per_night'].agg('mean').to_frame('price_mean_per_country')
df_country_outlier_id = df_country_price[df_country_price.price_mean_per_country > (df_country_price.price_mean_per_country.mean() + df_country_price.price_mean_per_country.std())]
df_country_price = df_country_price.drop(df_country_outlier_id.index.values.tolist())
df_country_outlier_id_2 = df_country_price[df_country_price.price_mean_per_country > (df_country_price.price_mean_per_country.mean() + 2*df_country_price.price_mean_per_country.std())]
df_country_outlier_id = pd.concat([df_country_outlier_id.reset_index(), df_country_outlier_id_2.reset_index()], ignore_index=True)
df_country_outlier_id.to_csv('data/price_outlier_country_id.csv')

'''
Drop price_usd outliers
'''
mask = df_all.prop_country_id.isin(df_country_outlier_id.prop_country_id.to_list())
df_all.price_usd[mask] /= df_all.srch_length_of_stay[mask]
drop_mask = (df_all.price_usd > 100000.0) | (df_all.price_usd == 0.0)
df_all.drop(df_all[drop_mask].index, inplace=True)

Inference of missing data

Impute orig_destination_distance, prop_location_score2, prop_review_score

In [5]:
'''
Impute orig_destination_distance by matrix between pairwise distances btw countries
'''
df_orig_destination_distance = df_all.groupby(['visitor_location_country_id', 'prop_country_id'])['orig_destination_distance'].agg('mean').to_frame('orig_destination_distance_mean')
df_orig_destination_distance = df_orig_destination_distance[~df_orig_destination_distance.orig_destination_distance_mean.isnull()].reset_index()
df_all = pd.merge(df_all,df_orig_destination_distance, on=['visitor_location_country_id', 'prop_country_id'], how='left')
df_all['orig_destination_distance'].fillna(df_all['orig_destination_distance_mean'], inplace=True)
df_all.drop(columns=['orig_destination_distance_mean'],inplace=True)
df_all.count()

Unnamed: 0                     9916106
srch_id                        9916106
date_time                      9916106
site_id                        9916106
visitor_location_country_id    9916106
visitor_hist_starrating        9916106
visitor_hist_adr_usd           9916106
prop_country_id                9916106
prop_id                        9916106
prop_starrating                9916106
prop_review_score              9901480
prop_brand_bool                9916106
prop_location_score1           9916106
prop_location_score2           7738075
prop_log_historical_price      9916106
position                       4957546
price_usd                      9916106
promotion_flag                 9916106
srch_destination_id            9916106
srch_length_of_stay            9916106
srch_booking_window            9916106
srch_adults_count              9916106
srch_children_count            9916106
srch_room_count                9916106
srch_saturday_night_bool       9916106
srch_query_affinity_score

In [6]:
'''
Impute prop_location_score2 by Linear Regression
'''
df_prop_location_score2 = df_all[['prop_location_score1', 'prop_location_score2', 'price_usd']]
df_prop_location_score2_train = df_prop_location_score2.dropna()
X = df_prop_location_score2_train[['prop_location_score1', 'price_usd']].to_numpy()
y = df_prop_location_score2_train['prop_location_score2'].to_numpy()
print('Train LR for prop_location_score2')
print(f'Shape of Train {X.shape} and label {y.shape}')
reg_prop_location_score2 = LinearRegression().fit(X,y)

pred = df_all[['prop_location_score1', 'price_usd']].to_numpy()
pred_y = reg_prop_location_score2.predict(pred)
df_all['prop_location_score2_pred'] = pred_y
df_all['prop_location_score2'].fillna(df_all['prop_location_score2_pred'], inplace=True)


'''
Impute prop_review_score by Linear Regression
'''
df_prop_review_score = df_all[['prop_review_score','prop_starrating', 'prop_location_score1', 'prop_location_score2', 'price_usd', 'prop_brand_bool']]
df_prop_review_score_train = df_prop_review_score.dropna()
X = df_prop_review_score_train.drop(columns=['prop_review_score']).to_numpy()
y = df_prop_review_score_train['prop_review_score'].to_numpy()
print('Train LR for prop_review_score')
print(f'Shape of Train {X.shape} and label {y.shape}')
reg_prop_review_score = LinearRegression().fit(X,y)

pred = df_all[['prop_starrating', 'prop_location_score1', 'prop_location_score2', 'price_usd', 'prop_brand_bool']].to_numpy()
pred_y = reg_prop_review_score.predict(pred)
df_all['prop_review_score_pred'] = pred_y
df_all['prop_review_score'].fillna(df_all['prop_review_score_pred'], inplace=True)

df_all.drop(columns=['prop_location_score2_pred', 'prop_review_score_pred'], inplace=True)

Train LR for prop_location_score2
Shape of Train (7738075, 2) and label (7738075,)
Train LR for prop_review_score
Shape of Train (9901480, 5) and label (9901480,)


Construction of new features

In [128]:
# def rename_df_col(df):
#     for col in df.columns:
#         col_name = '_'.join(col)
#         col_name = col_name.replace("(","").replace("'","").replace(")","").replace(" ","").replace(",","_")
#         df.rename({str(col):col_name},axis=1, inplace=True)
#         print(df.columns)
#     return df

# rename columns
def rename(col):
    if isinstance(col, tuple):
        col = '_'.join(str(c) for c in col)
    return col

def rename_df_col(df):
    df.columns = map(rename, df.columns)
    return df

In [129]:
'''
Made a new variable indicating whether the user making the querywas loacted in the country 
which was most common compared to the site (.jp, .com, etc.) or not
'''
# pandas according to two columns to find the max value
site_country_id = df_all.groupby(['visitor_location_country_id', 'site_id'])['site_id'].agg('count').to_frame('site_country_cnt')
site_country_id = site_country_id.loc[site_country_id.groupby('visitor_location_country_id').site_country_cnt.idxmax()]
site_country_id.to_csv('data/site_country_common_id.csv')

prop_feat_list = ['price_usd', 'prop_location_score1', 'prop_location_score2', 'prop_starrating', 'prop_review_score']
'''
Made a variable indicating the difference between the hotels price and the average
price in the country it is in, need to do the difference in the next section
'''
prop_avg_price_country = df_all.groupby('prop_country_id')[prop_feat_list].agg(['mean', 'std', 'median']).fillna(0)
prop_avg_price_country = rename_df_col(prop_avg_price_country)
prop_avg_price_country.add_suffix('_ctry').to_csv('data/prop_avg_price_country.csv')


'''
Made a variable indicating the count of search destination id to show the popularity of one place, need to join in the next section
'''
srch_des_stat = df_all.groupby('srch_destination_id')[prop_feat_list].agg(['mean', 'std', 'median']).fillna(0)
srch_des_cnt = df_all.groupby('srch_destination_id').agg({'srch_destination_id':'count'}).add_suffix('_cnt')
srch_des_stat = rename_df_col(srch_des_stat)
srch_des_cnt = srch_des_cnt.join(srch_des_stat.add_suffix('_dest'), on='srch_destination_id')
srch_des_cnt.to_csv('data/srch_des_cnt.csv')

'''
Made a variable indicating the amount of searches for that point of time
'''
df_all['date_time'] = pd.to_datetime(df_all.date_time, format='%Y-%m-%d %H:%M:%S')
df_all['year'] = df_all.date_time.dt.year
df_all['month'] = df_all.date_time.dt.month
df_all['day'] = df_all.date_time.dt.day
df_all['hour'] = df_all.date_time.dt.hour
df_all['dayofweek'] = df_all.date_time.dt.dayofweek

date_list = ['month', 'day', 'hour', 'dayofweek']
agg_list = ['click_bool', 'srch_id']
names = locals()
for i in date_list:
    names['df_srch_hist' + str(i) ] = df_all.groupby(['prop_id', i])[agg_list].agg('count').add_prefix('hist_').add_suffix(f'_{i}')
    names.get('df_srch_hist' + str(i)).to_csv(f'data/srch_hist_{i}.csv')

'''
Made a variable indicating how many searches there have been before per timepoint/year
'''
df_all['date_time_unix'] = (pd.to_datetime(df_all['date_time']) - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_srch_hist_timepoint = df_all.groupby(['prop_id', 'date_time_unix'])['prop_id'].sum().groupby(level=0).cumcount().to_frame('srch_hist_timepoint')
df_srch_hist_timepoint.to_csv('data/srch_hist_date_time_unix.csv')

In [130]:
'''
Made a lot of new variables
'''
# Aggregate the count of prop_id, promotion_flag and random_bool
hotel_cnt = df_all.groupby('prop_id')['prop_id'].agg('count').to_frame('srch_count')
hotel_pop = df_all.groupby('prop_id')[['promotion_flag', 'random_bool']].agg('sum')

# We made some new variables which were averages, medians and standard deviations over the property-id’s. 
# The rationale behind computing these summary statistics over a property was mostly to lower the noise in the data.
hotel_statistic = df_all.groupby('prop_id')[prop_feat_list].agg(['mean', 'std', 'median']).fillna(0)
hotel_statistic = rename_df_col(hotel_statistic)

# Variable mean of the position should exclude where the ordering was random
random_mask = df_train.random_bool == 0
position_feat = df_train[['prop_id', 'position']][random_mask].groupby('prop_id')['position'].agg(['mean','std','median']).add_prefix('position_').fillna(0)

df_hotel_all = hotel_cnt.join(hotel_pop, on='prop_id')
df_hotel_all = df_hotel_all.join(hotel_statistic.add_prefix('agg_'), on='prop_id')
df_hotel_all = df_hotel_all.join(position_feat, on='prop_id')
# df_hotel_all = fill_nan(df_hotel_all, 'median')
df_hotel_all.to_csv('data/hotel_data_all.csv')

In [131]:
df_hotel_all.columns

Index(['srch_count', 'promotion_flag', 'random_bool', 'agg_price_usd_mean',
       'agg_price_usd_std', 'agg_price_usd_median',
       'agg_prop_location_score1_mean', 'agg_prop_location_score1_std',
       'agg_prop_location_score1_median', 'agg_prop_location_score2_mean',
       'agg_prop_location_score2_std', 'agg_prop_location_score2_median',
       'agg_prop_starrating_mean', 'agg_prop_starrating_std',
       'agg_prop_starrating_median', 'agg_prop_review_score_mean',
       'agg_prop_review_score_std', 'agg_prop_review_score_median',
       'position_mean', 'position_std', 'position_median'],
      dtype='object')

# One-way count of the feature in only train dataset

In [132]:
df_train = pd.read_csv('data/training_cleaned.csv')
df_train = pd.merge(df_train,df_orig_destination_distance, on=['visitor_location_country_id', 'prop_country_id'], how='left')
df_train['orig_destination_distance'].fillna(df_train['orig_destination_distance_mean'], inplace=True)

'''
    Drop price_usd outliers for train set
'''
mask = df_train.prop_country_id.isin(df_country_outlier_id.prop_country_id.to_list())
df_train.price_usd[mask] /= df_train.srch_length_of_stay[mask]
drop_mask = (df_train.price_usd > 100000.0) | (df_train.price_usd == 0.0)
df_train.drop(df_train[drop_mask].index, inplace=True)

pred = df_train[['prop_location_score1', 'price_usd']].to_numpy()
pred_y = reg_prop_location_score2.predict(pred)
df_train['prop_location_score2_pred'] = pred_y
df_train['prop_location_score2'].fillna(df_train['prop_location_score2_pred'], inplace=True)

pred = df_train[['prop_starrating', 'prop_location_score1', 'prop_location_score2', 'price_usd', 'prop_brand_bool']].to_numpy()
pred_y = reg_prop_review_score.predict(pred)
df_train['prop_review_score_pred'] = pred_y
df_train['prop_review_score'].fillna(df_train['prop_review_score_pred'], inplace=True)

df_train.drop(columns=['orig_destination_distance_mean', 'prop_location_score2_pred', 'prop_review_score_pred'], inplace=True)

In [38]:
df_train.count()

Unnamed: 0                     4957546
srch_id                        4957546
date_time                      4957546
site_id                        4957546
visitor_location_country_id    4957546
visitor_hist_starrating        4957546
visitor_hist_adr_usd           4957546
prop_country_id                4957546
prop_id                        4957546
prop_starrating                4957546
prop_review_score              4957546
prop_brand_bool                4957546
prop_location_score1           4957546
prop_location_score2           4957546
prop_log_historical_price      4957546
position                       4957546
price_usd                      4957546
promotion_flag                 4957546
srch_destination_id            4957546
srch_length_of_stay            4957546
srch_booking_window            4957546
srch_adults_count              4957546
srch_children_count            4957546
srch_room_count                4957546
srch_saturday_night_bool       4957546
srch_query_affinity_score

In [133]:
# Possible impute alternative, beware: SLOWWWW
def KNN_impute(df):
    col_names = df.columns
    array = df.to_numpy()

    imputer = KNNImputer(n_neighbors=5)
    scaler = MinMaxScaler()
    scaler.fit(array)
    scaled = scaler.transform(array)

    imputed_scaled = imputer.fit_transform(scaled)
    imputed = scaler.inverse_transform(imputed_scaled)

    return pd.DataFrame(imputed, columns=col_names)

In [134]:
'''
Create dataframe with aggregated hotel data here
'''
# Aggregate by sum and count
hotel_pop = df_train.groupby('prop_id')[['booking_bool', 'click_bool']].agg('sum')
srch_query_affinity_score_mean = df_train.groupby('prop_id')['srch_query_affinity_score'].agg('mean')
srch_query_affinity_score_median = df_train.groupby('prop_id')['srch_query_affinity_score'].agg('median')

# Aggregate by mean
# hotel_feat = [feat for feat in list(df_train.columns) if 'prop' in feat] + ['srch_query_affinity_score'] 
df_hotel = pd.read_csv('data/hotel_data_all.csv')


df_hotel = df_hotel.join(hotel_pop, on='prop_id')
df_hotel = df_hotel.join(srch_query_affinity_score_mean, on='prop_id')
df_hotel = df_hotel.join(srch_query_affinity_score_median, on='prop_id', rsuffix='_median')
print(df_hotel.count())
knn_list = ['srch_query_affinity_score', 'srch_query_affinity_score_median', 'booking_bool', 'click_bool', 'promotion_flag', 'random_bool', 'srch_count', \
     'agg_prop_review_score_mean', 'agg_price_usd_mean', 'agg_prop_location_score1_mean', 'agg_prop_location_score2_mean', \
     'agg_price_usd_std', 'agg_prop_location_score2_std', 'position_mean', 'position_std', 'position_median']
df_knn = df_hotel[knn_list]

# # Apply log(x + 1) transform 
# df_hotel['booking_bool'] = np.log(df_hotel['booking_bool'] + 1)
# df_hotel['click_bool'] = np.log(df_hotel['click_bool'] + 1)
df_knn_imp = KNN_impute(df_knn)
print(df_knn_imp.count())

prop_id                             136885
srch_count                          136885
promotion_flag                      136885
random_bool                         136885
agg_price_usd_mean                  136885
agg_price_usd_std                   136885
agg_price_usd_median                136885
agg_prop_location_score1_mean       136885
agg_prop_location_score1_std        136885
agg_prop_location_score1_median     136885
agg_prop_location_score2_mean       136885
agg_prop_location_score2_std        136885
agg_prop_location_score2_median     136885
agg_prop_starrating_mean            136885
agg_prop_starrating_std             136885
agg_prop_starrating_median          136885
agg_prop_review_score_mean          136885
agg_prop_review_score_std           136885
agg_prop_review_score_median        136885
position_mean                       119745
position_std                        119745
position_median                     119745
booking_bool                        129111
click_bool 

In [41]:
df_knn_imp['prop_id'] = df_hotel['prop_id']
df_knn_imp.head()

,srch_query_affinity_score,srch_query_affinity_score_median,booking_bool,click_bool,promotion_flag,"('prop_starrating', 'mean')",random_bool,srch_count,"('prop_review_score', 'mean')","('price_usd', 'mean')","('prop_location_score1', 'mean')","('prop_location_score2', 'mean')","('price_usd', 'std')","('prop_location_score2', 'std')",position_mean,position_std,prop_id
0,-46.171467,-38.32070,0.0,1.0,0.0,2.0,27.0,123.0,0.0,94.960650,3.04,0.129791,10.161463,0.000002,30.891304,5.900012,1
1,-22.589767,-24.09040,1.0,1.0,0.0,0.0,3.0,29.0,4.0,97.149655,0.69,0.066790,4.394746,0.165269,16.400000,3.777124,2
2,-29.565475,-24.58075,2.0,2.0,1.0,3.0,40.0,172.0,3.5,117.853895,0.69,0.023659,96.052149,0.028234,20.238806,11.379260,3
3,-34.197487,-34.74040,1.0,1.0,12.0,5.0,20.0,42.0,4.5,485.960952,4.88,0.150732,177.329762,0.047842,16.727273,8.295672,4
4,-33.966717,-33.20363,0.0,2.0,3.0,0.0,30.0,52.0,0.0,605.083654,6.02,0.266439,225.930093,0.000055,24.384615,8.036774,5


In [42]:
df_hotel.head()

,prop_id,srch_count,promotion_flag,random_bool,"('prop_starrating', 'mean')","('prop_starrating', 'std')","('prop_starrating', 'median')","('prop_review_score', 'mean')","('prop_review_score', 'std')","('prop_review_score', 'median')",...,"('prop_location_score2', 'median')","('price_usd', 'mean')","('price_usd', 'std')","('price_usd', 'median')",position_mean,position_std,booking_bool,click_bool,srch_query_affinity_score,srch_query_affinity_score_median
0,1,123,0,27,2.0,0.0,2.0,0.0,0.0,0.0,...,0.129790,94.960650,10.161463,99.000,30.891304,5.900012,0.0,1.0,-46.171467,-38.32070
1,2,29,0,3,0.0,0.0,0.0,4.0,0.0,4.0,...,0.036100,97.149655,4.394746,97.000,16.400000,3.777124,1.0,1.0,-22.589767,-24.09040
2,3,172,1,40,3.0,0.0,3.0,3.5,0.0,3.5,...,0.021933,117.853895,96.052149,100.000,20.238806,11.379260,2.0,2.0,-29.565475,-24.58075
3,4,42,12,20,5.0,0.0,5.0,4.5,0.0,4.5,...,0.115500,485.960952,177.329762,434.725,16.727273,8.295672,1.0,1.0,NaN,NaN
4,5,52,3,30,0.0,0.0,0.0,0.0,0.0,0.0,...,0.266443,605.083654,225.930093,589.550,24.384615,8.036774,0.0,2.0,NaN,NaN


In [135]:
df_knn_imp['prop_id'] = df_hotel['prop_id']
df_knn_imp = df_knn_imp.set_index('prop_id')
drop_list = ['position_mean', 'position_std', 'position_median', 'srch_query_affinity_score', 'srch_query_affinity_score_median']
df_hotel_new = df_hotel.drop(columns=drop_list)
df_hotel_new = df_hotel_new.join(df_knn_imp[drop_list], on='prop_id')
for col in df_hotel_new.columns:
    prc_nan = df_hotel_new[col].isnull().sum() / len(df_hotel_new[col])
    print('Preserved {} with {:.2f} % missing'.format(col, prc_nan * 100))

df_hotel_cnt = df_train.groupby('prop_id')['prop_id'].agg('count').to_frame('srch_cnt_single')
df_hotel_new = df_hotel_new.join(df_hotel_cnt, on='prop_id').fillna(0)
df_hotel_new['click_prob'] = (df_hotel_new['click_bool'] + 0.0001)/(df_hotel_new['srch_cnt_single'] + 0.0001)
df_hotel_new['book_prob1'] = (df_hotel_new['booking_bool'] + 0.0001)/(df_hotel_new['click_bool'] + 0.0001)
df_hotel_new['book_prob2'] = (df_hotel_new['booking_bool'] + 0.0001)/(df_hotel_new['srch_cnt_single'] + 0.0001)
df_hotel_new.drop(columns=['srch_cnt_single'], inplace=True)
df_hotel_new.to_csv('data/hotel_data.csv')

Preserved prop_id with 0.00 % missing
Preserved srch_count with 0.00 % missing
Preserved promotion_flag with 0.00 % missing
Preserved random_bool with 0.00 % missing
Preserved agg_price_usd_mean with 0.00 % missing
Preserved agg_price_usd_std with 0.00 % missing
Preserved agg_price_usd_median with 0.00 % missing
Preserved agg_prop_location_score1_mean with 0.00 % missing
Preserved agg_prop_location_score1_std with 0.00 % missing
Preserved agg_prop_location_score1_median with 0.00 % missing
Preserved agg_prop_location_score2_mean with 0.00 % missing
Preserved agg_prop_location_score2_std with 0.00 % missing
Preserved agg_prop_location_score2_median with 0.00 % missing
Preserved agg_prop_starrating_mean with 0.00 % missing
Preserved agg_prop_starrating_std with 0.00 % missing
Preserved agg_prop_starrating_median with 0.00 % missing
Preserved agg_prop_review_score_mean with 0.00 % missing
Preserved agg_prop_review_score_std with 0.00 % missing
Preserved agg_prop_review_score_median with 0

Further Clean Train and Test set

In [2]:
def further_clean(df, dataset):

    if dataset != 'train':
        '''
        Drop price_usd outliers
        '''
        df_country_outlier_id = pd.read_csv('data/price_outlier_country_id.csv')
        mask = df.prop_country_id.isin(df_country_outlier_id.prop_country_id.to_list())
        df.price_usd[mask] /= df.srch_length_of_stay[mask]
    else:
        df_country_outlier_id = pd.read_csv('data/price_outlier_country_id.csv')
        mask = df.prop_country_id.isin(df_country_outlier_id.prop_country_id.to_list())
        df.price_usd[mask] /= df.srch_length_of_stay[mask]
        drop_mask = (df.price_usd > 100000.0) | (df.price_usd == 0.0)
        df.drop(df[drop_mask].index, inplace=True)

    df = pd.merge(df,df_orig_destination_distance, on=['visitor_location_country_id', 'prop_country_id'], how='left')
    df['orig_destination_distance'].fillna(df['orig_destination_distance_mean'], inplace=True)

    pred = df[['prop_location_score1', 'price_usd']].to_numpy()
    pred_y = reg_prop_location_score2.predict(pred)
    df['prop_location_score2_pred'] = pred_y
    df['prop_location_score2'].fillna(df['prop_location_score2_pred'], inplace=True)

    pred = df[['prop_starrating', 'prop_location_score1', 'prop_location_score2', 'price_usd', 'prop_brand_bool']].to_numpy()
    pred_y = reg_prop_review_score.predict(pred)
    df['prop_review_score_pred'] = pred_y
    df['prop_review_score'].fillna(df['prop_review_score_pred'], inplace=True)

    df.drop(columns=['orig_destination_distance_mean', 'prop_location_score2_pred', 'prop_review_score_pred', 'srch_query_affinity_score'], inplace=True)

    df.fillna(0, inplace=True)
    
    return df

In [7]:
df_train = pd.read_csv('data/training_cleaned.csv')
# clean train set
df_train_cleaned = further_clean(df_train, 'train')
print(df_train_cleaned.count())
df_train_cleaned.to_csv('data/train_cleaned_final.csv')

Unnamed: 0                     4957546
srch_id                        4957546
date_time                      4957546
site_id                        4957546
visitor_location_country_id    4957546
visitor_hist_starrating        4957546
visitor_hist_adr_usd           4957546
prop_country_id                4957546
prop_id                        4957546
prop_starrating                4957546
prop_review_score              4957546
prop_brand_bool                4957546
prop_location_score1           4957546
prop_location_score2           4957546
prop_log_historical_price      4957546
position                       4957546
price_usd                      4957546
promotion_flag                 4957546
srch_destination_id            4957546
srch_length_of_stay            4957546
srch_booking_window            4957546
srch_adults_count              4957546
srch_children_count            4957546
srch_room_count                4957546
srch_saturday_night_bool       4957546
orig_destination_distance

In [8]:
df_test = pd.read_csv('data/test_cleaned.csv')
# clean test set
df_test_cleaned = further_clean(df_test, 'test')
print(df_test_cleaned.count())
df_test_cleaned.to_csv('data/test_cleaned_final.csv')

Unnamed: 0                     4959183
srch_id                        4959183
date_time                      4959183
site_id                        4959183
visitor_location_country_id    4959183
visitor_hist_starrating        4959183
visitor_hist_adr_usd           4959183
prop_country_id                4959183
prop_id                        4959183
prop_starrating                4959183
prop_review_score              4959183
prop_brand_bool                4959183
prop_location_score1           4959183
prop_location_score2           4959183
prop_log_historical_price      4959183
price_usd                      4959183
promotion_flag                 4959183
srch_destination_id            4959183
srch_length_of_stay            4959183
srch_booking_window            4959183
srch_adults_count              4959183
srch_children_count            4959183
srch_room_count                4959183
srch_saturday_night_bool       4959183
orig_destination_distance      4959183
random_bool              

In [22]:
# check same srch_id same visitor_hist_adr_usd
df_train[~df_train['visitor_hist_adr_usd'].isnull()].head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
550,47,2013-01-14 22:28:43,14,100,4.31,180.75,137,6618,3,4.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,NaN,0
551,47,2013-01-14 22:28:43,14,100,4.31,180.75,137,11747,4,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
552,47,2013-01-14 22:28:43,14,100,4.31,180.75,137,18311,4,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
553,47,2013-01-14 22:28:43,14,100,4.31,180.75,137,19599,3,4.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0,NaN,0
554,47,2013-01-14 22:28:43,14,100,4.31,180.75,137,24002,4,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [8]:
for col in df.columns:
    prc_nan = df[col].isnull().sum() / len(df[col])
    if prc_nan != 0.0:
        print('Preserved {} with {:.2f} % missing'.format(col, prc_nan * 100))

Preserved srch_query_affinity_score with 93.60 % missing


In [1]:
'''
Balance dataset here. 
'''
# df_srch = df.join(df_hotel['srch_count'].to_frame('srch_count'), on='prop_id')
df_srch.query('srch_count > 4.75 | click_bool == 1')


X_train = df.to_numpy()[:, :-1]
y_train = df.to_numpy()[:, -1]

smt = SMOTETomek(ratio='auto')
X_smt, y_smt = smt.fit_sample(X_train, y_train)

NameError: name 'df_srch' is not defined